In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import ipyleaflet
import ipywidgets
import pandas as pd
import geopandas as gpd
from shapely.geometry import Polygon, Point
import datetime
import pathlib

In [4]:
from HidroData_optimized_v01 import get_stations

<Response [200]>


In [5]:
path_inventario = '../LHC_Hidroweb/invent2.csv'

In [6]:
df = pd.read_csv(path_inventario, engine='python', sep='\t', delimiter=';', parse_dates=['UltimaAtualizacao'])

In [7]:
df.columns

Index(['RegistroID', 'Importado', 'Temporario', 'Removido',
       'ImportadoRepetido', 'BaciaCodigo', 'SubBaciaCodigo', 'RioCodigo',
       'EstadoCodigo', 'MunicipioCodigo', 'ResponsavelCodigo',
       'ResponsavelUnidade', 'ResponsavelJurisdicao', 'OperadoraCodigo',
       'OperadoraUnidade', 'OperadoraSubUnidade', 'TipoEstacao', 'Codigo',
       'Nome', 'CodigoAdicional', 'Latitude', 'Longitude', 'Altitude',
       'AreaDrenagem', 'TipoEstacaoEscala', 'TipoEstacaoRegistradorNivel',
       'TipoEstacaoDescLiquida', 'TipoEstacaoSedimentos',
       'TipoEstacaoQualAgua', 'TipoEstacaoPluviometro',
       'TipoEstacaoRegistradorChuva', 'TipoEstacaoTanqueEvapo',
       'TipoEstacaoClimatologica', 'TipoEstacaoPiezometria',
       'TipoEstacaoTelemetrica', 'PeriodoEscalaInicio', 'PeriodoEscalaFim',
       'PeriodoRegistradorNivelInicio', 'PeriodoRegistradorNivelFim',
       'PeriodoDescLiquidaInicio', 'PeriodoDescLiquidaFim',
       'PeriodoSedimentosInicio', 'PeriodoSedimentosFim',
      

In [8]:
df[['Latitude', 'Longitude']] = df[['Latitude', 'Longitude']].apply(lambda x: x.str.replace(',','.'))
df['Latitude'] = df['Latitude'].astype('float')
df['Longitude'] = df['Longitude'].astype('float')

In [9]:
gdf = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df.Longitude,df.Latitude), crs='epsg:4674')

In [45]:
m01 = ipyleaflet.Map(zoom=2)

# Layers
try:
    path_shapefile = r'G:\Meu Drive\USP-SHS\Outros\Shapefile\Jaguaribe\Jaguaribe.shp'
    shape = gpd.read_file(path_shapefile)
    geo_data = ipyleaflet.GeoData(geo_dataframe=shape, name='Bacias',style={'color': 'black', 'fillColor': '#3366cc', 'opacity':0.05, 'weight':1.9, 'dashArray':'2', 'fillOpacity':0.6},
                   hover_style={'fillColor': 'red' , 'fillOpacity': 0.2})
    m01.add_layer(geo_data)
except:
    pass

## All points for HEATMAP
heatmap_all = ipyleaflet.Heatmap(locations=[tuple(r) for r in df[['Latitude', 'Longitude']].to_numpy()],radius=30, name='All point Heatmap')
m01.add_layer(heatmap_all)

radius_slider = ipywidgets.IntSlider(description='Radius', min=1, max=50, value=15)
ipywidgets.jslink((radius_slider,'value'),(heatmap_all,'radius'))
widget_control01 = ipyleaflet.WidgetControl(widget=radius_slider, position='bottomright')
m01.add_control(widget_control01)

## Last collection day
def date_location(_):
    heatmap_byLast.locations = [tuple(s) for s in df.loc[df['UltimaAtualizacao'] > date_slider.value, ['Latitude','Longitude']].to_numpy()]

heatmap_byLast = ipyleaflet.Heatmap(locations=[tuple(r) for r in df[['Latitude', 'Longitude']].to_numpy()],radius=30, name='By Date')
m01.add_layer(heatmap_byLast)

ipywidgets.jslink((radius_slider,'value'),(heatmap_byLast,'radius'))


date_slider = ipywidgets.SelectionSlider(options=pd.date_range(start='2000-01-01',end='2020-01-01', freq='M').to_numpy())
date_slider.observe(date_location,names='value')
widget_control02 = ipyleaflet.WidgetControl(widget=date_slider, position='topright')
m01.add_control(widget_control02)


teste = ipyleaflet.basemap_to_tiles(ipyleaflet.basemaps.NASAGIBS.ModisTerraTrueColorCR, "2020-01-30")


# Controles
## Layer
control_layer = ipyleaflet.LayersControl(position='topright')
m01.add_control(control_layer)

## Split Layer
# split_layer = ipyleaflet.SplitMapControl(left_layer=teste, right_layer=heatmap_byLast)
# m01.add_control(split_layer)

## Fullscreen
control_fullscreen = ipyleaflet.FullScreenControl()
m01.add_control(control_fullscreen)

## Desenho
control_desenho = ipyleaflet.DrawControl()
feature_collection = {'type': 'FeatureCollection',
                      'features': []}
def handle_draw(self, action, geo_json):
    """Do something with the GeoJSON when it's drawn on the map"""   
    feature_collection['features'].append(geo_json)

control_desenho.on_draw(handle_draw)
m01.add_control(control_desenho)

## Escala
control_escala = ipyleaflet.ScaleControl(position='bottomleft')
m01.add_control(control_escala)

## Select
box_layout = ipywidgets.Layout(display='flex',
                    flex_flow='column',
                    align_items='center',
                    width='100%')
box_layout2 = ipywidgets.Layout(display='flex',
                    flex_flow='column',
                    align_items='center',
                    width='100%')
control_selectDownload = ipywidgets.Dropdown(options=['Watershed', 'All', 'byDate'])
control_path = ipywidgets.Text(placeholder='Write your path HERE.')
control_buttom = ipywidgets.Button(description='Download')
control_rainOrflow = ipywidgets.RadioButtons(options=['Rain', 'Flow'])
control_progress = ipywidgets.IntProgress()
hbox1 = ipywidgets.HBox([control_rainOrflow,control_progress], layout=box_layout2)

vbox = ipywidgets.VBox([control_selectDownload, control_path, hbox1,control_buttom], layout=box_layout)
widget_control03 = ipyleaflet.WidgetControl(widget=vbox, position='bottomright')
m01.add_control(widget_control03)

def download_buttom(_):
    try:
        last_draw = feature_collection['features'][-1]['geometry']
        last_polygon = Polygon([(i[0], i[1]) for i in last_draw['coordinates'][0]])
        print(last_polygon)
    except:
        print('Nenhum polígono desenhado')
        
    if control_rainOrflow.value == 'Rain':
        a = 2
    if control_rainOrflow.value == 'Flow':
        a = 3
        
    if control_selectDownload.value == 'All':
#         sub = gdf.loc[(gdf['geometry'].within(last_polygon)),['Nome', 'Codigo']]
        code_list = gdf.loc[(gdf['geometry'].within(last_polygon)),'Codigo'].to_list()
        get_stations(list_codigos=code_list, typeData=a, folder_toDownload=control_path.value)

#         sub.to_csv(r'C:\Users\Usuario\Desktop\teste.csv')
        
    elif control_selectDownload.value == 'byDate':
#         sub = gdf.loc[(gdf['geometry'].within(last_polygon)) & (gdf['UltimaAtualizacao']>date_slider.value) ,['Nome', 'Codigo']]
        code_list = gdf.loc[(gdf['geometry'].within(last_polygon)) & (gdf['UltimaAtualizacao']>date_slider.value) ,'Codigo'].to_list()
        get_stations(list_codigos=code_list, typeData=a, folder_toDownload=control_path.value)


#         sub.to_csv(r'C:\Users\Usuario\Desktop\teste1.csv')
#         get_stations(list_codigos=code_list,typeData=a,folder_toDownload=control_path.value)
        
    elif control_selectDownload.value == 'Watershed':
        for i in shape['geometry']:
#             sub = gdf.loc[gdf['geometry'].within(i), ['Nome', 'Codigo']]
            code_list = gdf.loc[gdf['geometry'].within(i),  'Codigo'].to_list()
            get_stations(list_codigos=code_list, typeData=a, folder_toDownload=control_path.value)

#             sub.to_csv(r'C:\Users\Usuario\Desktop\teste1.csv')

            
        
control_buttom.on_click(download_buttom)
m01

[autoreload of HidroData_optimized_v01 failed: Traceback (most recent call last):
  File "C:\Users\Usuario\AppData\Roaming\Python\Python37\site-packages\IPython\extensions\autoreload.py", line 245, in check
    superreload(m, reload, self.old_objects)
  File "C:\Users\Usuario\AppData\Roaming\Python\Python37\site-packages\IPython\extensions\autoreload.py", line 394, in superreload
    module = reload(module)
  File "C:\Users\Usuario\Anaconda3\envs\LHC_Hidroweb\lib\imp.py", line 314, in reload
    return importlib.reload(module)
  File "C:\Users\Usuario\Anaconda3\envs\LHC_Hidroweb\lib\importlib\__init__.py", line 169, in reload
    _bootstrap._exec(spec, module)
  File "<frozen importlib._bootstrap>", line 630, in _exec
  File "<frozen importlib._bootstrap_external>", line 724, in exec_module
  File "<frozen importlib._bootstrap_external>", line 860, in get_code
  File "<frozen importlib._bootstrap_external>", line 791, in source_to_code
  File "<frozen importlib._bootstrap>", line 219, 

Map(center=[0.0, 0.0], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out_t…

In [38]:
import time

In [46]:
control_progress = ipywidgets.IntProgress(min=0, value=0,max=100)
display(control_progress)

IntProgress(value=0)

In [41]:
count = 0
max_count = 100
while count <= max_count:
    control_progress.value += 1
    time.sleep(0.1)
    count += 1

In [34]:
control_path.value

'C:\\Users\\Usuario\\Desktop\\lhc_hidroweb'

In [41]:
# try:
#     last_draw = feature_collection['features'][-1]['geometry']
#     last_polygon = Polygon([(i[0], i[1]) for i in last_draw['coordinates'][0]])
#     print(last_polygon)
# except:
#     print('Nenhum polígono desenhado')
    
# code_list = gdf.loc[(gdf['geometry'].within(last_polygon)),'Codigo'].to_list()
# get_stations(list_codigos=code_list,typeData=2,folder_toDownload=control_path.value)

Nenhum polígono desenhado
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
Estação sem dados
<Response [200]>
Estação sem dados
<Response [200]>
Estação sem dados
<Response [200]>
Estação sem dados
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
Estação sem dados
<Response [200]>
Estação sem dados
<Response [200]>
Estação sem dados
<Response [200]>
<Response [200]>
<Response [200]>
Estação sem dados


KeyboardInterrupt: 

In [ ]:
get_stations()